### Pipeline to run research to collect candidates

In [1]:
import sys
sys.path.append('../')

import pandas as pd
from datetime import datetime

import logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

from components.agents.research_agents import run_mixed_research_agents
from components.prompts.research_agents import FOCUS_AREAS
from data.content_saver import ContentSaver
from data.dedup import deduplicate_research

content_saver = ContentSaver(base_path="../data")

## 1. Using OpenAI, Anthropic and EXA web agents

In [2]:
FOCUS_AREAS.keys()

dict_keys(['reasoning_and_planning', 'agents_and_finance', 'agent_infrastructure', 'retrieval_and_embeddings', 'multimodal_and_generation'])

In [3]:
print(FOCUS_AREAS['reasoning_and_planning'])

Reasoning LLMs, chain-of-thought, inference-time compute, self-reflection, planning with LLMs, MCTS (Monte Carlo Tree Search) for language models, test-time scaling, hallucination reduction and detection, grounding, factuality


### Run

In [4]:
research_results = await run_mixed_research_agents(
    focus_areas=FOCUS_AREAS,
    content_saver=content_saver
)

2025-12-25 19:36:03,428 - components.agents.research_agents - INFO - Running 15 tasks
Research agents (mixed):   0%|          | 0/15 [00:00<?, ?it/s]2025-12-25 19:36:03,446 - components.agents.research_agents - INFO - Running Anthropic research agent for focus area: reasoning_and_planning
2025-12-25 19:36:03,462 - components.agents.research_agents - INFO - Running OpenAI research agent for focus area: retrieval_and_embeddings
2025-12-25 19:36:04,225 - components.agents.research_agents - INFO - Running Exa research agent for focus area: agent_infrastructure
2025-12-25 19:36:04,723 - components.agents.research_agents - INFO - Running OpenAI research agent for focus area: agents_and_finance
2025-12-25 19:36:04,729 - components.agents.research_agents - INFO - Running Anthropic research agent for focus area: retrieval_and_embeddings
2025-12-25 19:36:04,735 - components.agents.research_agents - INFO - Running OpenAI research agent for focus area: agent_infrastructure
2025-12-25 19:36:04,737 

### Deduplicate based on URL

In [5]:
# Parse the research results into a flat list of records
records = []
for key, result in research_results.items():
    focus_area, llm_provider = key.split(' --- ')
    for item in result.items:
        records.append({
            'focus_area': focus_area,
            'provider': llm_provider,
            'url': item.url,
            'title': item.title,
            'source': item.source,
            'published': item.published,
            'relevance': item.relevance,
            'date_added': datetime.now().strftime("%Y-%m-%d")
        })

# Create DataFrame
df = pd.DataFrame(records)

print(f"There are {len(df)} ResearchItems present in the research results")

There are 445 ResearchItems present in the research results


In [6]:
# Deduplicate based on URL (keep first occurrence)
print(f"df.shape: {df.shape}")
df = df.drop_duplicates(subset='url', keep='first')
print(f"df.shape: {df.shape}")

df.shape: (445, 8)
df.shape: (428, 8)


### Add to existing collection

In [8]:
result = deduplicate_research(
    new_df=df,
    main_csv="../data/research_items.csv",
    save=True
)

print(f"Added {result['new_added']} new items")
print(f"Skipped {result['skipped']} duplicates")
print(f"Total in DB: {result['total_after']}")

2025-12-25 19:45:25,970 - data.dedup - INFO - Loaded 1337 existing items from ../data/research_items.csv
2025-12-25 19:45:25,972 - data.dedup - INFO - New items: 0, Skipped (duplicates): 428
2025-12-25 19:45:25,992 - data.dedup - INFO - Saved 1337 items to ../data/research_items.csv


Added 0 new items
Skipped 428 duplicates
Total in DB: 1337


## 2. Using direct arXiv API

In [10]:
from components.tools import ArxivClient, ARXIV_CATEGORIES, ARXIV_KEYWORDS
arxiv = ArxivClient()

In [11]:
arxiv_results = arxiv.search_cats_and_kws_both(
    categories=ARXIV_CATEGORIES,
    keywords=ARXIV_KEYWORDS,
    max_results_cats=200,
    max_results_kws=150,
    last_n_days=None
)

2025-12-25 19:47:23,107 - components.tools.arxiv - INFO - Found 200 entries for submitted categories
2025-12-25 19:47:33,207 - components.tools.arxiv - INFO - Found 150 entries for submitted keywords
2025-12-25 19:47:33,209 - components.tools.arxiv - INFO - Found 350 total entries


In [12]:
len(arxiv_results.items)

350

Dedup

In [13]:
# Convert to DataFrame (same as web agents)
records = []
for item in arxiv_results.items:
    records.append({
        'focus_area': 'arxiv',
        'provider': 'arxiv',
        'url': item.url,
        'title': item.title,
        'source': item.source,
        'published': item.published,
        'relevance': item.relevance,
        'date_added': datetime.now().strftime("%Y-%m-%d")
    })

df_arxiv = pd.DataFrame(records)
print(f"There are {len(df_arxiv)} ResearchItems present in the research results")

There are 350 ResearchItems present in the research results


In [14]:
# Deduplicate based on URL (keep first occurrence)
print(f"df.shape: {df_arxiv.shape}")
df_arxiv = df_arxiv.drop_duplicates(subset='url', keep='first')
print(f"df.shape: {df_arxiv.shape}")

df.shape: (350, 8)
df.shape: (272, 8)


Add to existing collection

In [16]:
result = deduplicate_research(
    new_df=df_arxiv,
    main_csv="../data/research_items.csv",
    save=True
)

print(f"Added {result['new_added']} new items")
print(f"Skipped {result['skipped']} duplicates")
print(f"Total in DB: {result['total_after']}")

2025-12-25 19:48:02,541 - data.dedup - INFO - Loaded 1356 existing items from ../data/research_items.csv
2025-12-25 19:48:02,543 - data.dedup - INFO - New items: 0, Skipped (duplicates): 272
2025-12-25 19:48:02,561 - data.dedup - INFO - Saved 1356 items to ../data/research_items.csv


Added 0 new items
Skipped 272 duplicates
Total in DB: 1356


### Check research items csv

In [17]:
recent_df = pd.read_csv("../data/research_items.csv")
recent_df.head()

,focus_area,provider,url,title,source,published,relevance,date_added
0,dummy1,openai,https://dummy1.com,dummy1,OpenAI Blog,2025-12-01,dummy example 1,2025-12-20
1,dummy2,anthropic,https://dummy2.com,dummy2,arXiv,2025-12-15,dummy example 2,2025-12-21
2,reasoning_agent,openai,https://openai.com/index/gpt-5-2-codex,Introducing GPT-5.2-Codex,OpenAI blog,2025-12-18,Official release of an agentic coding model em...,2025-12-24
3,reasoning_agent,openai,https://openai.com/index/introducing-gpt-5-2/,Introducing GPT-5.2,OpenAI blog,2025-12-11,Details GPT-5.2 “Thinking/Pro” modes and API r...,2025-12-24
4,reasoning_agent,openai,https://blog.google/products/gemini/gemini-3/,Introducing Gemini 3: our most intelligent mod...,Google Blog (Gemini/DeepMind),2025-11-18,Announces Gemini 3 with “thinking”/Deep Think ...,2025-12-24


In [18]:
provider_counts = recent_df['provider'].value_counts()
print("Absolute counts:")
print(provider_counts)
print("\nNormalized (proportions):")
print(recent_df['provider'].value_counts(normalize=True))

Absolute counts:
provider
arxiv        481
openai       359
anthropic    321
exa          195
Name: count, dtype: int64

Normalized (proportions):
provider
arxiv        0.354720
openai       0.264749
anthropic    0.236726
exa          0.143805
Name: proportion, dtype: float64
